# Data Properties


For many sequential decision making problems we look for some key patterns in the data

* Markov property
* A consistent reward or cost
* Actions being effective in contributing to the reward or affecting the Environment
* Seeing if there is a consistent way that actions are picked

We have a few helper visualisations to help these are `markovian_matrix` and
`normalized_markovian_matrix`.

**Pre-requisite**: this example requires stable-baselines3. To quickly install this library, you may
uncoment and execute the next cell. Note that the term `'gym>=...'` prevents `stable-baselines3`
from downgrading `gym` to a version incompatible with `a2rl`.

In [ ]:
# %pip install stable-baselines3 'gym>=0.23.1,<0.26.0'

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import my_nb_path  # isort: skip
import a2rl as wi  # isort: skip
import os

import gym
from IPython.display import Markdown
from stable_baselines3 import A2C, DQN, SAC
from stable_baselines3.common.base_class import BaseAlgorithm

import a2rl.nbtools  # Enable color outputs when rich is installed.
from a2rl.utils import NotMDPDataError, assert_mdp, data_generator_simple, plot_information

## Helper Functions

In [ ]:
def discretized_sample_dataset(dataset_name: str, n_bins=50) -> wi.WiDataFrame:
    """Discretized a sample dataset.

    Args:
        dataset_name: name of the sample dataset.

    Returns:
        Whatif dataframe.

    See Also
    --------
    list_sample_datasets
    """
    dirname = wi.sample_dataset_path(dataset_name)
    tokeniser = wi.DiscreteTokenizer(n_bins=n_bins)
    df = tokeniser.fit_transform(wi.read_csv_dataset(dirname))
    return df


def data_generator_gym(
    env_name: str = "Taxi-v3",
    trainer: type[BaseAlgorithm] = A2C,
    training_steps: int = 10000,
    capture_steps: int = 1000,
) -> wi.WiDataFrame:
    """Generate a :class:`a2rl.WiDataFrame` from any well-defined OpenAi gym.
    An agent is trained first for ``training_steps``. Then, capture ``capture_steps`` from the
    trained agent.
    Args:
        env_name: Name of the gym environment.
        trainer: An underlying generator algorithm that supports discrete actions, such as
            :class:`stable_baselines3.dqn.DQN` or :class:`stable_baselines3.a2c.A2C`. Raise an error
            when passing a trainer that does not support discrete actions, such as
            :class:`stable_baselines3.sac.SAC`.
        training_steps: The number of steps to train the generator.
        capture_steps: The number of steps to capture.
    Returns:
        A2RL data frame.
    """
    env = gym.make(env_name, render_mode=None)
    model = trainer(policy="MlpPolicy", env=env, verbose=False)
    model.learn(total_timesteps=training_steps)

    cap_env = wi.TransitionRecorder(env)
    model.set_env(cap_env)
    model.learn(total_timesteps=capture_steps)

    tokeniser = wi.DiscreteTokenizer(n_bins=50)
    df = tokeniser.fit_transform(cap_env.df)

    return df


def test_gym_generator():
    import pytest

    gym_data = data_generator_gym(env_name="Taxi-v3", trainer=DQN)
    assert isinstance(gym_data, wi.WiDataFrame)

    with pytest.raises(AssertionError, match=r"Discrete(.*) was provided"):
        gym_data = data_generator_gym(env_name="MountainCar-v0", trainer=SAC)

In [ ]:
lags = 10  # Same as assert_mdp()'s default.

################################################################################
# To run in fast mode, set env var NOTEBOOK_FAST_RUN=1 prior to starting Jupyter
################################################################################
if os.environ.get("NOTEBOOK_FAST_RUN", "0") != "0":
    lags = 5
    display(
        Markdown(
            '<p style="color:firebrick; background-color:yellow; font-weight:bold">'
            "NOTE: notebook runs in fast mode. Use only 5 lags. Results may differ."
        )
    )
################################################################################

## Data Inspection

In the offline setting we are restricted only to data. A2RL offers three ways to generate some:

1. The load-and-discretize workflow <- The main one. See `discretized_sample_dataset()`.

2. `data_generator_gym` to load data interations between a trained agent and a gym environment <- This is for testing and research

3. `data_generator_simple` to generate sample data with different properties <- Also for testing and research

### Synthetic Data


Create Markov property and then add random actions (random policy) that affect the states. 

In [ ]:
offline_data = data_generator_simple(
    markov_order=1,
    reward_function=False,
    action_effect=True,
    policy=False,
)

try:
    assert_mdp(offline_data, lags=lags)
except NotMDPDataError as e:
    print("Continue this example despite MDP check errors:\n", e)

plot_information(offline_data, lags=lags);

Use higher order Markov property and effective actions, and add a reward function that is related to the state and action

In [ ]:
offline_data = data_generator_simple(
    markov_order=2,
    reward_function=True,
    action_effect=True,
    policy=False,
)

try:
    assert_mdp(offline_data, lags=lags)
except NotMDPDataError as e:
    print("Continue this example despite MDP check errors:\n", e)

plot_information(offline_data, lags=lags);

### OpenAi gym environment with known MDP

Use an agent that is not trained very much on Taxi dataset and see how's the data looks like.

In [ ]:
%%time
from stable_baselines3 import DQN

gym_data = data_generator_gym(
    env_name="Taxi-v3",
    trainer=DQN,
    training_steps=10000,
    capture_steps=100,
)

try:
    assert_mdp(offline_data, lags=lags)
except NotMDPDataError as e:
    print("Continue this example despite MDP check errors:\n", e)

plot_information(gym_data, lags=lags);

### Chiller Data

In [ ]:
%%time

df_chiller = discretized_sample_dataset("chiller", n_bins=10)
try:
    assert_mdp(df_chiller, lags=lags)
except NotMDPDataError as e:
    print("Continue this example despite MDP check errors:\n", e)

plot_information(df_chiller, lags=lags);